In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from sklearn.model_selection import train_test_split
import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.
from keras.models import Sequential
from keras.preprocessing import sequence
from keras.models import Model, Input
from keras.layers import Dense, Embedding, GlobalMaxPooling1D, Conv1D, Dropout, Activation, SpatialDropout1D, Bidirectional, CuDNNLSTM
from keras.preprocessing.text import Tokenizer
from keras.optimizers import Adam
from keras import backend as K

from keras.initializers import Constant

In [ ]:
def get_glove_embedding():
    _embeddings_index = {}
    f = open(os.path.join('../input/','glove-global-vectors-for-word-representation','glove.twitter.27B.100d.txt'))
    # f = open(os.path.join('../input/','glove6b100dtxt','glove.6B.100d.txt'))
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        _embeddings_index[word] = coefs
    f.close() 
    
    return _embeddings_index
    
embeddings_index = get_glove_embedding()

print('Found %s word vectors.' % len(embeddings_index))


In [ ]:
#test=pd.read_csv('../input/test.csv')
train=pd.read_csv(os.path.join('../input/','jigsaw-unintended-bias-in-toxicity-classification','train.csv'))
#X_train = train["comment_text"].values


#X_test = test["comment_text"].values
train.head()

In [ ]:
def try_underSampling(train):
    
    #Prepare for under sampling 
    religion = train[(train.religion == 1)]
    print("target religion : ",religion.shape)
    ethnicity = train[(train.ethnicity == 1)]
    print("ethnicity: ",ethnicity.shape)
    sexualOrientation = train[(train.sexualOrientation == 1)]
    print("sexualOrientation: ",sexualOrientation.shape)
    gender = train[(train.sex == 1)]
    print("gender: ",gender.shape)
    no_hatespeech = train[(train.religion == 0) | (train.ethnicity == 0) | (train.sexualOrientation == 0) | (train.sex == 0)]
    print("No hatespeech : ",no_hatespeech.shape, no_hatespeech.tail())
    all_together = train[(train.religion == 1) | (train.ethnicity == 1) | (train.sexualOrientation == 1) | (train.sex == 1)]
    
    religion = religion.sample(1189, replace=True)
    ethnicity = ethnicity.sample(1189, replace=True)
    sexualOrientation = sexualOrientation.sample(1189, replace=True)
    gender = gender.sample(1189, replace=True)
    no_hatespeech = no_hatespeech.sample(1189, replace=True)
    all_together = all_together.sample(1189, replace=True)

    print("New target religion : ",religion.shape)
    print("New ethnicity: ",ethnicity.shape)
    print("New sexualOrientation: ",sexualOrientation.shape)
    print("New gender: ",gender.shape)
    print("New No hatespeech : ",no_hatespeech.shape)
    print("All : ",all_together.shape)
    
    train_set = pd.concat([religion, ethnicity, sexualOrientation, gender, no_hatespeech, all_together])
    train_set = train_set[['id', 'target', 'comment_text', 'religion','ethnicity','sexualOrientation', 'sex']]

    return train_set

In [ ]:
def one_hot_encode_by_label(dataset, new_col, col_list, threshold_val=0.1):
    
    for col in col_list:
        dataset[new_col] = (
            dataset[col] > threshold_val
        )*1.0  
#         dataset[new_col] = (
#             dataset[col] > threshold_val and not dataset[['funny', 'wow', 'sad', 'likes']].any()
#         )*1.0  
    return dataset


def one_hot_encode_neutral(dataset, new_col, col_list, threshold_val=0.2):
    for col in col_list:
        dataset[new_col] = (
            dataset[col] < threshold_val
        )*0
    return dataset

#
religions = [
    'atheist',
    'buddhist',
    'christian',
    'hindu',
    'jewish',
    'muslim',
    'other_religion'
]
train = one_hot_encode_by_label(train, 'religion', religions)

#
ethnicity = [
    'asian',
    'black',
    'latino',
    'white',
    'other_race_or_ethnicity'
]
train = one_hot_encode_by_label(train, 'ethnicity', ethnicity)

# 
sexualOrientation = [
    'bisexual',
    'heterosexual',
    'homosexual_gay_or_lesbian',
    'transgender',
    'other_sexual_orientation'    
]
train = one_hot_encode_by_label(train, 'sexualOrientation', sexualOrientation)

#
sex = [
    'male',
    'female',
    'other_gender' 
]

train = one_hot_encode_by_label(train, 'sex', sex)

#
neutral = [
    'asian',
    'black',
    'latino',
    'white',
    'other_race_or_ethnicity',
    'asian',
    'black',
    'latino',
    'white',
    'other_race_or_ethnicity',
    'bisexual',
    'heterosexual',
    'homosexual_gay_or_lesbian',
    'transgender',
    'other_sexual_orientation',
    'male',
    'female',
    'other_gender' 
]

#train = one_hot_encode_neutral(train, 'neutral', neutral)

train=train[(train.religion == 1) | (train.ethnicity == 1) | (train.sexualOrientation == 1) | (train.sex == 1)]
print("Take everythin with hatespeech: ", train.shape)

#train[ train['religion'] == 1 ].tail()
#train[['id', 'target', 'comment_text', 'religion','ethnicity','sexualOrientation', 'sex']]

train = try_underSampling(train)

train.tail()



In [ ]:
X = train['comment_text']
y = train[['religion','ethnicity','sexualOrientation', 'sex']]


X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

In [ ]:
y_train = y_train[[ 'religion','ethnicity','sexualOrientation', 'sex']].values
y_test = y_test[[ 'religion','ethnicity','sexualOrientation', 'sex']].values

In [ ]:
max_features = 20000  # number of words we want to keep
maxlen = 200  # max length of the comments in the model
batch_size = 64  # batch size for the model
embedding_dims = 100  # dimension of the hidden variable, i.e. the embedding dimension
epochs = 10


In [ ]:
tok = Tokenizer(num_words=max_features)

X_train_org = X_train
X_test_org = X_test

tok.fit_on_texts(list(X_train) + list(X_test))
x_train = tok.texts_to_sequences(X_train)
x_test = tok.texts_to_sequences(X_test)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')
print('Average train sequence length: {}'.format(np.mean(list(map(len, x_train)), dtype=int)))
print('Average test sequence length: {}'.format(np.mean(list(map(len, x_test)), dtype=int)))

In [ ]:
word_index = tok.word_index
print('Found %s unique tokens.' % len(word_index))

num_words = min(max_features, len(word_index)) + 1
print(num_words)

# first create a matrix of zeros, this is our embedding matrix
embedding_matrix = np.zeros((num_words, embedding_dims))

# for each word in out tokenizer lets try to find that work in our w2v model
for word, i in word_index.items():
    if i > max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # we found the word - add that words vector to the matrix
        embedding_matrix[i] = embedding_vector
    else:
        # doesn't exist, assign a random vector
        embedding_matrix[i] = np.random.randn(embedding_dims)

In [ ]:
y_train=y_train[:1209265,:]
y_test=y_test[:1209265,:]

In [ ]:
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print(x_train)

In [ ]:
def create_model(max_features, embedding_dims):
    print('Build model...')
    model = Sequential()

    # we start off with an efficient embedding layer which maps
    # our vocab indices into embedding_dims dimensions
    model.add(Embedding(max_features,
                        embedding_dims,
                        input_length=maxlen))
    model.add(Dropout(0.2))

    # we add a Convolution1D, which will learn filters
    # word group filters of size filter_length:
    model.add(Conv1D(filters=3,
                     kernel_size=100,
                     padding='valid',
                     activation='relu',
                     strides=1))
    # we use max pooling:
    model.add(GlobalMaxPooling1D())

    # We add a vanilla hidden layer:
    model.add(Dense(100))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))

    # We project onto a single unit output layer, and squash it with a sigmoid:
    model.add(Dense(4))
    model.add(Activation('sigmoid'))
    
    return model

In [ ]:
def create_model_org(max_features, embedding_dims):
    comment_input = Input((maxlen,))

    # we start off with an efficient embedding layer which maps
    # our vocab indices into embedding_dims dimensions
    comment_emb = Embedding(max_features, embedding_dims, input_length=maxlen, 
                            embeddings_initializer="uniform")(comment_input)
    
    # we add a GlobalMaxPooling1D, which will extract features from the embeddings
    # of all words in the comment
    pooling = GlobalMaxPooling1D()(comment_emb)
    
    # We add a hidden layer:
    h=Dense(5, activation='relu')(pooling)
    #model.add(Activation('relu'))

    # We project onto a three-unit output layer, and squash it with a sigmoid:
    output = Dense(4, activation='sigmoid')(h)
    return Model(inputs=comment_input, outputs=output)

In [ ]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:

model1 = create_model_org(max_features, embedding_dims)
model1.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy',f1])
model1.summary()

hist = model1.fit(x_train, y_train, batch_size=batch_size, verbose=1, epochs=epochs, validation_split=0.1)

In [ ]:
# lstm bidirectional with glove embeding model creation 
def create_model_org_lstm_bidir_glove():
    model = Sequential()
    model.add(Embedding(
        num_words, 
        embedding_dims, 
        input_length=maxlen,
        embeddings_initializer=Constant(embedding_matrix),
        trainable=True        
    ))
    model.add(SpatialDropout1D(0.2))
    model.add(Bidirectional(CuDNNLSTM(64, return_sequences=True)))
    model.add(Bidirectional(CuDNNLSTM(32)))
    model.add(Dropout(0.25))
    model.add(Dense(units=4, activation='softmax'))

    return model

In [ ]:
#  lstm bidirectional with glove embeding compilation and fitting
model2 = create_model_org_lstm_bidir_glove()
model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', f1])
model2.summary()

hist = model2.fit(x_train, y_train, batch_size=batch_size, verbose=1, epochs=epochs, validation_split=0.1)


In [ ]:
y_precit = model2.predict(x_test)
len(x_test), y_test, len(y_test), y_precit

In [ ]:
def get_confusion_matrix(model, x_test,y_true):
    y_pred = model.predict(x_test)
    #print(y_pred)
    y_pred = (y_pred > 0.5) #greater than 0.50 on scale 0 to 1
    #print(y_pred)

    from sklearn.metrics import classification_report
    print(classification_report(y_true, y_pred))

In [ ]:
# 
dat_test = pd.DataFrame(["stupid atheist", "kill muslims", "Kill hindus"], columns = ['comment_text'])['comment_text']
dat_test_test2 = tok.texts_to_sequences(dat_test)
x_test_2 = sequence.pad_sequences(dat_test_test2, maxlen=maxlen)
y_pred = model2.predict(x_test_2)
y_pred_df = pd.DataFrame(y_pred)
print('relgion (1)', y_pred_df.idxmax(axis=1))

dat_test = pd.DataFrame(["Destroy americans", "Dirty indian", "Illiterate mexican"], columns = ['comment_text'])['comment_text']
dat_test_test2 = tok.texts_to_sequences(dat_test)
x_test_2 = sequence.pad_sequences(dat_test_test2, maxlen=maxlen)
y_pred = model2.predict(x_test_2)
y_pred_df = pd.DataFrame(y_pred)
print('ethnicity (2)', y_pred_df.idxmax(axis=1))


dat_test = pd.DataFrame(["Bloddy bisexual", "Homosexuals are garbage", "Kill homosexual"], columns = ['comment_text'])['comment_text']
dat_test_test2 = tok.texts_to_sequences(dat_test)
x_test_2 = sequence.pad_sequences(dat_test_test2, maxlen=maxlen)
y_pred = model2.predict(x_test_2)
y_pred_df = pd.DataFrame(y_pred)
print('sexual orientation (3)', y_pred_df.idxmax(axis=1))

dat_test = pd.DataFrame(["This bitch is nuts. Who would read a book by a woman.", "Kill woman society", "Bloddy female group"], columns = ['comment_text'])['comment_text']
dat_test_test2 = tok.texts_to_sequences(dat_test)
x_test_2 = sequence.pad_sequences(dat_test_test2, maxlen=maxlen)
y_pred1 = model2.predict(x_test_2)
y_pred_df = pd.DataFrame(y_pred1)
print('gender (4)', y_pred_df.idxmax(axis=1))

In [ ]:
get_confusion_matrix(model2, x_test,y_test)

In [ ]:
get_confusion_matrix(model1, x_test,y_test)

In [ ]:
#NB implementation
def create_naive_bayes(train):
    #implementation of naive bayes

    from sklearn.metrics import classification_report
    from sklearn.metrics import accuracy_score

    from sklearn.model_selection import train_test_split
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.feature_extraction.text import TfidfTransformer
    from sklearn.naive_bayes import MultinomialNB

    from sklearn.feature_extraction.text import TfidfVectorizer
    tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
    features = tfidf.fit_transform(train.comment_text).toarray()

    train["tags"] =0
    #train.tail()
    train["tags"] = train.apply(lambda x: x['tags'] if x['religion']==0 else x['religion']*0, axis=1)
    train["tags"] = train.apply(lambda x: x['tags'] if x['ethnicity']==0 else x['ethnicity']*1, axis=1)
    train["tags"] = train.apply(lambda x: x['tags'] if x['sexualOrientation']==0 else x['sexualOrientation']*2, axis=1)
    train["tags"] = train.apply(lambda x: x['tags'] if x['sex']==0 else x['sex']*3, axis=1) 
    labels = train["tags"]

    print(features.shape, labels.shape)

    X = train['comment_text']
    y = train['tags']
    train['comment_text'], train['tags']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)


    from sklearn.naive_bayes import MultinomialNB
    from sklearn.pipeline import Pipeline
    from sklearn.feature_extraction.text import TfidfTransformer

    nb = Pipeline([('vect', CountVectorizer()),
                   ('tfidf', TfidfTransformer()),
                   ('clf', MultinomialNB()),
                  ])
    nb.fit(X_train, y_train)

    from sklearn.metrics import classification_report
    y_pred3 = nb.predict(X_test)
    from sklearn.metrics import accuracy_score
    print('accuracy %s' % accuracy_score(y_pred3, y_test))

    my_tags = ['religion','ethnicity','sexualOrientation','sex']

    print(classification_report(y_test, y_pred3,target_names=my_tags))

    print(nb.predict(X_test) ,"\n real \n", y_test.values)
    
    return nb


In [ ]:
nb = create_naive_bayes(train)

In [ ]:
# Testing naive bayes with examples
dat_test = pd.DataFrame(["stupid atheist", "kill muslims", "Kill hindus"], columns = ['comment_text'])['comment_text']
y_pred = nb.predict(dat_test)
y_pred_df = pd.DataFrame(y_pred)
print('relgion (0)', y_pred_df.values)

dat_test = pd.DataFrame(["Destroy americans", "Dirty indian", "Illiterate mexican"], columns = ['comment_text'])['comment_text']
y_pred = nb.predict(dat_test)
y_pred_df = pd.DataFrame(y_pred)
print('ethnicity (1)', y_pred_df.values)


dat_test = pd.DataFrame(["Bloddy bisexual", "Homosexuals are garbage", "Kill homosexual"], columns = ['comment_text'])['comment_text']
y_pred = nb.predict(dat_test)
y_pred_df = pd.DataFrame(y_pred)
print('sexual orientation (2)', y_pred_df.values)

dat_test = pd.DataFrame(["This bitch is nuts. Who would read a book by a woman.", "Kill woman society", "Bloddy female group"], columns = ['comment_text'])['comment_text']
y_pred = nb.predict(dat_test)
y_pred_df = pd.DataFrame(y_pred)
print('gender (3)', y_pred_df.values)

In [ ]:

#Implementation of SVM starts here

def create_SVM(train):
    
    from sklearn.svm import SVC
    from sklearn.metrics import confusion_matrix,classification_report
    from sklearn.model_selection import cross_val_score, GridSearchCV

    from sklearn.model_selection import train_test_split
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.feature_extraction.text import TfidfTransformer


    X = train['comment_text']
    y = train['tags']
    train['comment_text'], train['tags']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

    svm_model = Pipeline([('vect', CountVectorizer()),
                   ('tfidf', TfidfTransformer()),
                   ('clf', SVC()),
                  ])

    # Performing CV to tune parameters for best SVM fit 
    #svm_model = GridSearchCV(SVC(), params_grid, cv=5)
    #svm_model.fit(X_train_tfidf, y_train)

    svm_model.fit(X_train, y_train)
    return svm_model

In [ ]:
svm_model = create_SVM(train)
dat_test = pd.DataFrame(["stupid atheist", "kill muslims", "Kill hindus"], columns = ['comment_text'])['comment_text']
svm_pred = svm_model.predict(X_test)

In [ ]:
my_tags = ['religion','ethnicity','sexualOrientation','sex']

print(classification_report(y_test, svm_pred,target_names=my_tags))

In [ ]:
dat_test = pd.DataFrame(["stupid atheist", "kill muslims", "Kill hindus"], columns = ['comment_text'])['comment_text']
y_pred = svm_model.predict(dat_test)
y_pred_df = pd.DataFrame(y_pred)
print('relgion (0)', y_pred_df.values)

dat_test = pd.DataFrame(["Destroy americans", "Dirty indian", "Illiterate mexican"], columns = ['comment_text'])['comment_text']
y_pred = svm_model.predict(dat_test)
y_pred_df = pd.DataFrame(y_pred)
print('ethnicity (1)', y_pred_df.values)


dat_test = pd.DataFrame(["Bloddy bisexual", "Homosexuals are garbage", "Kill homosexual"], columns = ['comment_text'])['comment_text']
y_pred = svm_model.predict(dat_test)
y_pred_df = pd.DataFrame(y_pred)
print('sexual orientation (2)', y_pred_df.values)

dat_test = pd.DataFrame(["This bitch is nuts. Who would read a book by a woman.", "Kill woman society", "Bloddy female group"], columns = ['comment_text'])['comment_text']
y_pred = svm_model.predict(dat_test)
y_pred_df = pd.DataFrame(y_pred)
print('gender (3)', y_pred_df.values)